# Project of Translation from Archaic to Modern Italian

In [1]:
# Import Datases to work with Transformers by Hugging-Face
import torch
# Imports for Transformers

from datasets import Dataset
from transformers import AutoTokenizer  # Datasets
import huggingface_hub
from datasets.features import Value, Features

import pandas as pd



# Promposed Models

### Prompt Learning
* google/gemma-3-4b-it (LLM) for context learning - use ChatPrompt 🚀  
* meta-llama/Llama-3.2-3B   another LLM for context learning - use ChatPrompt 🦙 
### Fine Tunning
* sapienzanlp/Minerva-1B-base-v1.0  fine-tunned for transaltion task(LMM) - use MinervaPrompt (same for fine-tunning)  🇮🇹 

### Native Machine Translation Systems
* Helsinki-NLP/opus-mt-itc-itc (Machine Translation) for translation task - use OpusPrompt 🏆  
* facebook/nllb-200-3.3B (Machine Translation) another translation machine  🤖 

### Other Task
* models--prometheus-eval--prometheus-7b-v2.0 has jugde - use PrometheusPrompt 🔥 

# System Setup 🖥️

### Additional Dependencies 🐍

In [2]:
#!bash install.sh

### Hugging Face 🤗

In [3]:
TOKEN = ""
#huggingface_hub.login(token=TOKEN)

# Globals Variables

In [4]:
DATASET = "dataset.csv"
FEATURES = Features(
    {
        "Author": Value(dtype="string"),
        "Date": Value(dtype="string"),
        "Region": Value(dtype="string"),
        "Sentence": Value(dtype="string")
    }
)
NET = "google/gemma-3-1b-it"
BS = 1
PROMPT = "seqchat"
DEVICE = ('cuda' if torch.cuda.is_available() else "cpu")

## Prompts


### Prompts for Prompt Learning (LLM)

In [5]:
tokenizer = None
max_length=1024

In [6]:
def seqchat_map(examples):
    chat = tokenizer.apply_chat_template([
    [
        {"role": "system",   "content": "Rispondendo in Italiano riscrivi Le Frasi in Italiano Antico in Italiano Moderno sequendo le indicazioni dello 'user' e rispondendo precisamente alle richieste senza dare spiegazioni"},
        {"role": "user",     "content": "Frasi in Italiano Antico: 'Nel mezzo del cammin di vita nostra  mi ritrovai per una selva oscura ché la diritta via era smarrita', sotituisci i termini poco utilizzati o errati"},
        {"role": "assistant","content": "Nuova Frase : 'A metà del cammino della vita nostra mi sono ritrovato in un selva buia e avevo smarrito la giusta strada'"},
        {"role": "user",     "content": "Riordina le parole in modo che la frase risulti più scorrevole"},
        {"role": "assistant","content": "Nuova Frase : 'A metà del cammino della nostra vita mi sono ritrovato in un selva buia e avevo smarrito la giusta strada'"},
        {"role": "user",     "content": "Migliora il significato della frase"},
        {"role": "assistant","content": "Nuova Frase : 'A metà del cammino della mia vita (mezza età) mi sono ritrovato in un selva buia e avevo perso la giusta strada'"},

        {"role": "user",     "content": f"Frasi in Italiano Antico: '{example}', sotituisci i termini poco utilizzati o errati"},
        {"role": "assistant", "content": ""}
    ] for example in examples["Sentence"]], 
        tokenize=True,
        return_dict=True,
        padding=True,         # adds zeros to make all sequences the same length
        truncation=True,      # cuts sequences that are too long
        max_length=max_length,       # (optional) max length of the sequences
        return_tensors="pt")

    return chat
def chat_map(examples):
    chat = tokenizer.apply_chat_template([
    [
        {"role": "system",   "content": "Sei un traduttore esperto di Italiano Antico. Devi tradurre in modo conciso i brani che ti vengono data dallo 'user'"},
        {"role": "user",     "content": "Traduci 'La corte era in gran fermento.' in Italiano Moderno"},
        {"role": "assistant","content": "Italiano Antico: 'La corte era in gran fermento.' Italiano Moderno: 'La corte era molto agitata.'"},
        {"role": "user",      "content": f"Traduci '{example}' in Italiano Moderno"},
        {"role": "assistant", "content": ""}
    ] for example in examples["Sentence"]], 
        tokenize=True,
        return_dict=True,
        padding=True,         # adds zeros to make all sequences the same length
        truncation=True,      # cuts sequences that are too long
        max_length=max_length,       # (optional) max length of the sequences
        return_tensors="pt")

    return chat

def gemma_1b_map(examples):
    chat = tokenizer.apply_chat_template([
    [
        {"role": "system",   "content": "Sei un traduttore esperto di Italiano Antico "},
        {"role": "user",     "content": "Traduci 'La corte era in gran fermento.' in Italiano Moderno"},
        {"role": "assistant","content": "Italiano Antico: 'La corte era in gran fermento.' Italiano Moderno: 'La corte era molto agitata.'"},
        {"role": "user",      "content": f"Traduci '{example}' in Italiano Moderno"},
        {"role": "assistant", "content": ""}
    ] for example in examples["Sentence"]], 
        tokenize=True,
        return_dict=True,
        padding=True,         # adds zeros to make all sequences the same length
        truncation=True,      # cuts sequences that are too long
        max_length=max_length,       # (optional) max length of the sequences
        return_tensors="pt")

    return chat

### Prompts for Machine Translation Systems

### Prompts For Fine-Tunned Models

### Prompts for Other Task (Judge etc...)

In [7]:
def ft5_std_map(examples):
    return tokenizer(
        [f"Traduci dal volgare all’italiano moderno: {example}" for example in examples["Sentence"]],  
        padding=True, 
        max_length=128,
        )

def mask_std_map(examples):
    return tokenizer(
        [f"Old Italian: {example} Modern Italian [MASK]" for example in examples['Sentence']],  
        padding=True, 
        max_length=128)

def minerva_map(examples):
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(
        [f"Traduci dal volgare all’italiano moderno: {example}" for example in examples["Sentence"]],  
        padding=True, 
        max_length=128,
        )

def style_map(examples):
    return tokenizer(
        [f"The following sentence represents an example from the Dolce Stil Novo (sweet new style) literary movement, developed in the 13th and 14th century in Italy: {example} Translate it to modern Italian: " for example in examples["Sentence"]],
        padding=True, 
        max_length=128)

def period_region_map(examples):
    return tokenizer(
        [f"This sentence {example['Sentence']} was written in {example['Date']}, in the {example['Region']} region. Translate it to Modern Italian" for example in examples],
        padding=True,
        max_length=128)

def author_map(examples):
    return tokenizer(
        [f"This sentence: {example['Sentence']} was written by {example['Author']}. Translate it to Modern Italian" for example in examples],
        padding=True,
        max_length=128)

def question_map(examples):
    return tokenizer(
        [f"Puoi riscrivere questa frase: {example} in uno stile più colloquiale?" for example in examples['Sentence']],
        padding=True,
        max_length=128)

### Prompt Selection

In [8]:
# Switch to select the mapping function based on the prompt type

match PROMPT:
    case "ft5_std":
        tr = ft5_std_map
    case "gemma-1b":
        tr = gemma_1b_map
    case "mask_std":
        tr = mask_std_map
    case "style":
        tr = style_map
    case "period_region":
        tr = period_region_map
    case "author":
        tr = author_map
    case "question":
        tr = question_map
    case "minerva":
        tr = minerva_map
    case "llma-1b":
        tr = llma_1b_map
    case "chat":
        tr = chat_map
    case "seqchat":
        tr = seqchat_map    
    case _:
        raise ValueError("Unknown prompt type")

### Network Selection (Configuration + Tokenizer)

In [9]:
# Switch to select the network and load the appropriate model and tokenizer
match NET:
    
    case "google/flan-t5-small" | "google-t5/t5-small" | "google/mt5-small" | "google/flan-t5-large":
        from transformers import T5Tokenizer, T5ForConditionalGeneration
        tokenizer = T5Tokenizer.from_pretrained(NET)
        model = T5ForConditionalGeneration.from_pretrained(NET, device_map=DEVICE, torch_dtype=torch.float16)
     

        params = {
            
            "max_new_tokens": 120, # max number of new tokens to generate
            "do_sample":True,      # enables sampling for more diverse outputs
            "top_k":50,            # diversity increase by controlling the candidate words
            "top_p":0.90,          # nucleus sampling for further control over variety
            "temperature":1.0,     # reduces randomness and increases coherence
            "repetition_penalty":1.0,  # penalizza ripetizioni
            "num_return_sequences":10,  # number of generated responses
            "pad_token_id":tokenizer.eos_token_id  # avoids warning if padding token is missing
        }
        
    case "google/mt5-base":
        from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
        tokenizer = AutoTokenizer.from_pretrained(NET)
        model = AutoModelForSeq2SeqLM.from_pretrained(NET, device_map=DEVICE, torch_dtype=torch.float16)
 

        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,          
            "top_p":0.90,          
            "temperature":1.0,  
            "repetition_penalty":1.0,
            "num_return_sequences":10, 
            "pad_token_id":tokenizer.eos_token_id 
        }

    case "google/gemma-3-1b-it" | "google/gemma-3-4b-it" | "google/gemma-3n-E4B-it-litert-preview":
        from transformers import BitsAndBytesConfig, Gemma3ForCausalLM, AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(NET)
        quantization_config = BitsAndBytesConfig(load_in_8bit=True)
        model = Gemma3ForCausalLM.from_pretrained(NET, device_map=DEVICE, quantization_config=quantization_config)
    
        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,     
            "top_p":0.90,  
            "temperature":1.0,     
            "repetition_penalty":1.0,
            "num_return_sequences":10,  
            "pad_token_id":tokenizer.eos_token_id 
        }
    
    case "FacebookAI/xlm-roberta-base":
        
        from transformers import AutoTokenizer, AutoModelForMaskedLM
        tokenizer = AutoTokenizer.from_pretrained(NET)
        model = AutoModelForMaskedLM.from_pretrained(NET)

        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,            
            "top_p":0.90,         
            "temperature":1.0,    
            "repetition_penalty":1.0, 
            "num_return_sequences":10,  
            "pad_token_id":tokenizer.eos_token_id 
        }
    case "sapienzanlp/Minerva-1B-base-v1.0":
        
        from transformers import AutoTokenizer, AutoModelForCausalLM
        tokenizer = AutoTokenizer.from_pretrained(NET)
        model = AutoModelForCausalLM.from_pretrained(NET, device_map=DEVICE)
  
        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,            # aumento della diversità controllando le parole candidate
            "top_p":0.90,          # campionamento nucleus per ulteriori controlli sulla varietà
            "temperature":1.0,     # riduce la casualità e aumenta la coerenza
            "repetition_penalty":1.0,  # penalizza ripetizioni
            "num_return_sequences":10,  # numero di risposte generate
            "pad_token_id":tokenizer.eos_token_id  # evita warning se manca un token di padding
        }
    case "meta-llama/Llama-3.2-3B-Instruct-QLORA_INT4_EO8":
        from transformers import LlamaForCausalLM, AutoTokenizer
        from transformers import BitsAndBytesConfig
        quantization = BitsAndBytesConfig(load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(NET, padding_side='left', use_fast=False)
        
        model = LlamaForCausalLM.from_pretrained(NET, device_map=DEVICE, quantization_config=quantization)
        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,            # aumento della diversità controllando le parole candidate
            "top_p":0.90,          # campionamento nucleus per ulteriori controlli sulla varietà
            "temperature":1.0,     # riduce la casualità e aumenta la coerenza
            "repetition_penalty":1.0,  # penalizza ripetizioni
            "num_return_sequences":10,  # numero di risposte generate
            "pad_token_id":tokenizer.eos_token_id  # evita warning se manca un token di padding
        }
    
    case "mistralai/Mistral-7B-Instruct-v0.2":
        from transformers import AutoTokenizer, AutoModelForCausalLM
        from transformers import BitsAndBytesConfig
        quantization = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
        tokenizer = AutoTokenizer.from_pretrained(NET, padding_side='left')
        tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained(NET, device_map=DEVICE, quantization_config=quantization, torch_dtype=torch.bfloat16, attn_implementation="sdpa")
        params = {
            
            "max_new_tokens": 512,
            "do_sample":True,
            "top_k":10,            # aumento della diversità controllando le parole candidate
            "top_p":0.90,          # campionamento nucleus per ulteriori controlli sulla varietà
            "temperature":0.85,     # riduce la casualità e aumenta la coerenza
            "repetition_penalty":1.0,  # penalizza ripetizioni
            #"num_return_sequences":10,  # numero di risposte generate
            "pad_token_id":tokenizer.eos_token_id  # evita warning se manca un token di padding
        }

    case _:
        raise Exception(f"Rete {NET} non testabile")
        

# Dataset

In [10]:
hf = Dataset.from_csv(DATASET, features=FEATURES).shuffle(seed=42)

In [11]:
tokenized = hf.map(tr, batched=True)

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [12]:
print(tokenized.column_names)
for idx, s in enumerate(tokenized.take(5), 1):
    print(f"sample n°{idx}: {s}")
    print(f"Decode ids: {tokenizer.decode(s["input_ids"], attention_mask=s["attention_mask"], skip_special_tokens=True)}")
    

['Author', 'Date', 'Region', 'Sentence', 'input_ids', 'attention_mask']
sample n°1: {'Author': 'Guido da Pisa', 'Date': '1337', 'Region': 'tosc.', 'Sentence': "Ed ecco di subito tutta questa turba degli uccelli si levò a volo dietro all'aquila", 'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 105, 2364, 107, 99510, 18630, 4362, 528, 168248, 221537, 152039, 1834, 2102, 4166, 528, 168248, 5307, 2486, 528, 168248, 13806, 236748, 4659, 4362, 674, 3114, 14910, 39584, 756, 2364, 236789, 545, 168606, 4362, 92431, 7854, 224420, 32060, 30253, 94073, 14910, 108, 4255, 4166, 528, 168248, 5307, 2486, 236787, 756, 65962, 83781, 1162, 3322, 1387, 1001, 26627, 52887, 138, 1327, 148440, 1389, 810, 1985, 6025, 2919, 228046, 170582, 759, 6557, 39404, 4323, 6933, 1406, 516, 58103, 963, 67993, 509, 8461, 1287, 858, 91953, 17817, 189079, 512, 3683, 1625, 106, 107, 105, 4368, 107, 43868, 6636, 2102, 781, 1017, 756, 236776, 119963, 116

In [13]:
from utils import evaluate_and_save


df = evaluate_and_save(
    model=model,
    judge=None,
    tokenizer=tokenizer,
    tokenized_dataset=tokenized,
    output_prefix="./Translation model",
    device=DEVICE,
    batch_size=BS,
    generate_params=params
)

0it [00:00, ?it/s]

["user\nRispondendo in Italiano riscrivi Le Frasi in Italiano Antico in Italiano Moderno sequendo le indicazioni dello 'user' e rispondendo precisamente alle richieste senza dare spiegazioni\n\nFrasi in Italiano Antico: 'Nel mezzo del cammin di vita nostra  mi ritrovai per una selva oscura ché la diritta via era smarrita', sotituisci i termini poco utilizzati o errati\nmodel\nNuova Frase : 'A metà del cammino della vita nostra mi sono ritrovato in un selva buia e avevo smarrito la giusta strada'\nuser\nRiordina le parole in modo che la frase risulti più scorrevole\nmodel\nNuova Frase : 'A metà del cammino della nostra vita mi sono ritrovato in un selva buia e avevo smarrito la giusta strada'\nuser\nMigliora il significato della frase\nmodel\nNuova Frase : 'A metà del cammino della mia vita (mezza età) mi sono ritrovato in un selva buia e avevo perso la giusta strada'\nuser\nFrasi in Italiano Antico: 'Ed ecco di subito tutta questa turba degli uccelli si levò a volo dietro all'aquila', 

AttributeError: 'NoneType' object has no attribute 'judge'